# SDDP y sus variantes

## Andrés Ferragut

## Grupo MATE - Universidad ORT Uruguay

### Setiembre 2021

## Agenda

* Repasar el algotitmo SDDP.
* Discutir el caso azar-decisión y decisión-azar.
* Presentación de un caso decisión-azar (newsvendor problem).
* Variantes que no requieren modelo de ruido (learning).
* El caso multi-bien y el caso multi-etapa.

## Dynamic Programming y azar-decisión.

Consideremos el problema clásico de programación dinámica, con estado $x_k$, control $u_k$ y horizonte finito $T$ sin costo final. Sea $w_k$ el ruido de entrada (por ahora $iid$):

$$\min E\left[\sum_{k=0}^{T-1} g_k(u_k,x_k,w_k)\right]$$

con $x_{k+1} = f_k(u_k,x_k,w_k)$.

### Iteración de Bellman

<img style="float:right; width: 200px; padding-left:1em" src="bellman.jpg" />

El problema anterior se resuelve mediante la iteración clásica de Bellman:

* Condición inicial: $V_T (x_T) \equiv 0$

* Backward recursion:

$$V_k(x) = \min_{u_k} E_{w_k}\left[g_k(u_k,x,w_k) + V_{k+1}(x_{k+1})\right]$$

con $x_{k+1} = f_k(u_k,x,w_k)$.

> **Nota:** este es el caso clásico, donde suponemos que el optimizador no conoce $w_k$ antes de tomar la decisión $u_k$. Llamamos a este caso *decisión-azar*.

## El caso azar-decisión

Supongamos que el optimizador conoce $w_k$ antes de tomar la decisión $u_k$. Podemos formular el problema como uno clásico extendiendo el espacio de estados:

$$\tilde{x}_k = \begin{pmatrix} x_k \\ w_k \end{pmatrix},$$

con dinámica:

$$\tilde{x}_{k+1} = \begin{pmatrix} f_k(u_k,x_k,w_k) \\ w_{k+1} \end{pmatrix}.$$

Observemos que en este caso el ruido que entra involucra a $w_{k+1}$ en la dinámica (o sea $\tilde{w}_k = w_{k+1}$).

### Bellman déjà-vu

* Backward recursion original:

$$\begin{align*}
\tilde{V}_k(\tilde{x}_k) = \tilde{V}_k(x_k,w_k) &= \min_{u_k} E_{\tilde{w}_k}\left[g_k(u_k,x_k,w_k) + \tilde{V}_{k+1}(\tilde{x}_{k+1})\right] \\
                     &= \min_{u_k} \left[g_k(u_k,x_k,w_k) + E_{w_{k+1}}[\tilde{V}_{k+1}(x_{k+1}, w_{k+1})]\right].
\end{align*}$$

* Backward recursion en dos pasos:

$$\begin{align*}
V_T(x) &\equiv 0 \\
\tilde{V}_k (x_k,w_k) &= \min_{u_k} \left[g_k(u_k,x_k,w_k) + V_{k+1}(x_{k+1})\right] \\
V_{k}(x_k) &= E_{w_{k}}\left[\tilde{V}_{k}(x_{k}, w_{k})]\right].
\end{align*}$$


### SDDP

Esto lleva a la iteración básica de SDDP: supongamos que $w_k$ tiene soporte finito, entonces $E_{w_{k+1}}$ se puede calcular promediando. Si además sustituimos $\tilde{V}_k$ por una aproximación por hiperplanos, obtendremos promediando una aproximación por hiperplanos de $V_k$.

**Algoritmo:**
 
1. $\hat{V}_T(x) \equiv 0$ 
2. Calculo $\hat{V}_k (x_k,w_k)=\min_{u_k,x} [g_k(u_k,x_k,w_k)+ \hat{V}_{k+1}(x_{k+1})]$, forzando $x=x_k$. 
 
3. Con el multiplicador de $x=x_k$ calculo un corte para $\hat{V}_k (x_k,w_k)$.
4. Promediando los cortes en $w_k$ obtengo un corte para $\hat{V}_{k}(x_k)$.


**Punto clave:** el punto $x_{k+1}$ solo depende de $u_k$ (el control a determinar). $w_k$ y $x_k$ son dados.

## El caso decisión-azar

Volviendo a la recursión original:

$$V_k(x_k) = \min_{u_k} E_{w_k}\left[g_k(u_k,x_k,w_k) + V_{k+1}(x_{k+1})\right]$$

con $x_{k+1} = f_k(u_k,x_k,w_k)$.

no parece posible realizar la misma descomposición en dos pasos de la recursión aislando la esperanza.

**Problema:** el punto $x_{k+1}$ sigue siendo indeterminado, depende del ruido aun no observado. Por lo tanto, es difícil establecer donde pondríamos un corte para $V_{k+1}(x_{k+1})$.

### ¿Y si el ruido entra solo en la dinámica?

La recursión queda:

$$V_k(x_k) = \min_{u_k} \left\{g_k(u_k,x_k) + E_{w_k}[V_{k+1}(x_{k+1})]\right\}$$

con $x_{k+1} = f_k(u_k,x_k,w_k)$.

**Problema:** el punto $x_{k+1}$ sigue siendo indeterminado...

### Q-factors

¿Qué ocurre si nos pasamos a los factores $Q$? 

Sea:

$$Q_k(x_k,u_k) = E_{w_k}[g_k(u_k,x_k,w_k) + V_{k+1}(x_{k+1})]$$

con $x_{k+1} = f_k(u_k,x_k,w_k)$.

Recordemos que $V_k(x_k) = \min_{u_k} Q_k(x_k,u_k)$.

### Bellman déjà-qu

Recursión de Bellman para $Q-$factors:

$$Q_k(x_k,u_k) = E_{w_k} \left[g_k(u_k,x_k,w_k) + \min_{u_{k+1}} Q_{k+1}(x_{k+1},u_{k+1}) \right]$$


### Intento de SDDP con Q-factors

Como $x_k$ y $u_k$ son dados podemos escribir este último término como:

$$E_{w_k}[\min_{u,x,u_{k+1}} g_k(u,x,w_k) + Q_{k+1}(x_{k+1},u_{k+1})],$$

donde la optimización adentro de la esperanza tiene como restricciones:

* $u=u_k$
* $x=x_k$
* $x_{k+1} = f_k(x,u,w_k)$.

Por lo tanto, de los múltiplicadores de $u_k$ y $x_k$ podemos sacar un hiperplano de soporte para:

$$\min_{u_{k+1}}  g_k(u_k,x_k,w_k) + Q_{k+1}(x_{k+1},u_{k+1})$$

Promediando en los $w_k$ generamos un hiperplano de soporte para la esperanza.

### Algoritmo SDDP para Q-factors (SDQL, yo lo bauticé, avisen si existe)

1. Inicializo $Q_T \equiv 0$.
2. Calculo $\hat{Q}_k (x_k,u_k,w_k)=\min_{u,x,u'} [g_k(u_k,x_k,w_k)+ \hat{Q}_{k+1}(x_{k+1},u')]$, forzando:
   * $u=u_k$,
   * $x=x_k$,
   * $x_{k+1} = f_k(x,u,w_k)$,
   
  para cada posible $w_k$.
3. Con los multiplicadores de $x=x_k$ y $u=u_k$, calculo un corte para $\hat{Q}_k (x_k,u_k, w_k)$.
4. Promediando los cortes en $w_k$ obtengo un corte para $\hat{Q}_{k}(x_k,u_k)$.


### Caso interesante (relacionado al newsvendor)

En varios problemas de control de stock, el valor está asociado al "estado post-decisión". En términos matemáticos, la dinámica es afín en $x$ y $u$ y por ejemplo el valor solo depende de una combinación de $x$ y $u$. A su vez el costo del paso no depende de $w_k$.

>**Ejemplo:** Si $x$ es el stock al final del día y $u$ es la reposición de la noche, debemos enfrentar la demanda del día con $x+u$ y el valor futuro debería ser el mismo para un mismo $x+u$.

Dicho de otro modo:

$$Q_k(x_k,u_k) = J_k(x_k+u_k)$$

para alguna $J$ apropiada.

**Nota:** en este caso deberíamos poder reducir los multiplicadores a uno solo. Todavía no me salió pero espero que el problema de newsvendor ilustre algo de esto.

## Newsvendor problem

El problema clásico de "newsvendor" o "canillita" hace referencia al siguiente problema de compra en dos etapas:

 * Se debe proveer un bien a una demanda aleatoria $w$ con distribución conocida.
 * Se puede realizar una compra "day-ahead" a precio $p$ para tener stock.
 * Si la demanda supera a la compra day-ahead, se debe comprar a precio $q>p$ (precio spot) el mismo día para compensar.
 
**Pregunta:** ¿Cuánto debo comprar el día anterior?

### Solución al newsvendor problem

Es fácil ver que el costo total de lo anterior, para una compra $u$ day-ahead y una demanda $w$ está dado por:

$$c(u,w) = pu + q (w-u)^+$$

De donde el costo medio es:

$$C(u) = pu + q E[(w-u)^+]$$

Puede verse que:

$$\frac{d}{du} E[(w-u)^+] = E\left[ \frac{d}{du} (w-u)^+\right] = E\left[-\mathbf{1}_{\{u<w\}}\right] = -P(u < w)$$

De donde:
$$C'(u) = p - qP(u < w) = p - q (1-F_w(u))$$

y el $u^*$ está caracterizado por:
$$u^*: 1-F_w(u^*) = \frac{p}{q}.$$

**Ejemplo:** si $p=q/2$, tengo que comprar la mediana de la distribución.

### Newsvendor y programación dinámica

Uno puede escribir el problema anterior en dos etapas como:

**2a. etapa (spot):**

$$\min_{u_1} q u_1$$

sujeto a:

* $x_1 + u_1 \geqslant w$,
* $u_1\geqslant 0$

Siendo $x_1$ el stock al comenzar el día, es decir $x_0+u_0$.

Sea $V(x_1) = E[\min_{u_1} q u_1]$ sujeto a las restricciones anteriores, entonces:

**1a. etapa (day-ahead):**

$$\min_{u_0} p u_0 + V(x_1)$$

sujeto a $x_1=x_0+u_0$. En este caso $V$ juega las veces de "costo futuro".

### Ejemplo: demanda uniforme

En el caso de que $w\sim U[0,\theta]$, $V(x) = E[(w-x)^+]$ se puede calcular explícitamente y da:

$$V(x) = q\frac{(\theta - x)^2}{2\theta}$$

In [1]:
using Plots, LaTeXStrings
default(size=(900,500), legendfontsize=10)

θ=100
x=(0:100)
q=2.0

55
665jh+u

plot(x,V.(x,q,θ), label=L"V(x), q=%$q, \theta=%$θ", xlabel=L"x", lw=2)

LoadError: UndefVarError: jh not defined

### Solución mediante SDDP

**Idea:** generar una aproximación por hiperplanos de $V(x)$.

**Forward step:** Dada una aproximación $V^{(l-1)}(x)$ resuelvo:

$$\min_{u_0} \left\{ p u_0 + V^{(l-1)}(x_1) \right\}$$

con $x_0$ dado y $x_1 = x_0+u_0$. Obtengo el $x_1^{(l)}$ (estado a explorar).

**Backward step:** Dado $x_1^{(l)}$ resuelvo para cada ruido $w$:

$$\beta(w,x_1^{(l)}) = \min_{u_1} q u_1$$

sujeto a:
* $x_1^{(l)} + u_1 \geqslant w \quad [\lambda(w,x_1^{(l)})]$,
* $u_1\geqslant 0$

Entonces $\beta(x_1^{(l)}) = E_w[\beta(w,x_1^{(l)})]$ y $\lambda(x_1^{(l)}) = E_w[\lambda(w,x_1^{(l)})]$ definen un corte para la función $V$ en $x_1^{(l)}$. Agrego este corte y construyo $V^{(l)}$.

### Ejemplo:

Tomemos demanda uniforme con $\theta=100$ y discreticemos los ruidos en $\{0,1,\ldots,100\}$. $p=1.0$, $q=2.0$. En este caso, el control óptimo es comprar $50$.

In [2]:
using Distributions, JuMP, Gurobi, Plots, ProgressMeter, StatsBase
gurobi_env = Gurobi.Env();

Academic license - for non-commercial use only - expires 2021-11-06


In [3]:
function piecewise_linear(x,cuts)
    
    return maximum([cut[1]+cut[2]*x for cut in cuts])
end

piecewise_linear (generic function with 1 method)

In [4]:
p = 1.0 #day ahead price
q = 2.0 #same day price

#condicion inicial de stock
stock_ini=0.0;

In [5]:
#array de vectores de cuts. Arranca en la lower bound
#cuts = beta + lambda x_1 -> [beta,lambda]
cuts = [[0.0,0.0]];

niter=20

@showprogress for i=1:niter
    
    #Forward Step
    model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))
    @variable(model,u0>=0);
    @variable(model,x0>=0);
    @variable(model,z);

    for i=1:length(cuts)
        cut=cuts[i]
        @constraint(model,z>=cut[1]+cut[2]*(x0+u0));
    end

    fix_x = @constraint(model,x0.==stock_ini);

    @objective(model,Min,p*u0+z);

    optimize!(model)

    x1_value=value.(u0)+value.(x0);

    #BACKWARD STEP
    #For every noise
    local_cuts = [];

    for w=0:100 #demanda uniforme theta=100
        model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))

        @variable(model,u1>=0);
        fix_x = @constraint(model,x1_value-w+u1.>=0);

        @objective(model,Min,q*u1);

        optimize!(model)

        beta = objective_value(model);
        lambda = -dual.(fix_x)

        push!(local_cuts,[beta-lambda*x1_value;lambda])
    end

    new_cut = mean(local_cuts)
    push!(cuts,new_cut);
    
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:18


In [6]:
#Ploteo los cuts obtenidos
x=(0:100)

anim=nothing
anim = @animate for i=1:length(cuts)
    pl=plot(;xlim=(0,100),ylim=(-10,100))
    plot!(pl,x,V.(x,q,θ),lw=2 , label=L"V(x)")
    plot!(pl,x,piecewise_linear.(x,Ref(cuts[1:i])),lw=2, label=L"V^{(l)}(x), l=%$i")
    plot!(pl,x,piecewise_linear.(x,Ref([cuts[i]])), lw=1, ls=:dash, label="corte agregado")
    plot!(pl,xlabel=L"x")
end

gif(anim,"tmp.gif",fps=1)

LoadError: UndefVarError: V not defined

### Resuelvo el problema de stock

Con la aproximación de $V$ obtenida puedo resolver el problema forward una última vez para hallar el control óptimo:

In [7]:
model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))
@variable(model,u0>=0);
@variable(model,x0>=0);
@variable(model,z);

for i=1:length(cuts)
    cut=cuts[i]
    @constraint(model,z>=cut[1]+cut[2]*(x0+u0));
end

fix_x = @constraint(model,x0.==stock_ini);

@objective(model,Min,p*u0+z);

optimize!(model)

x1_value=value.(u0)+value.(x0);
println("Stock day ahead: $x1_value.")
println("Costo estimado: $(objective_value(model)).")


Stock day ahead: 49.99999999999982.
Costo estimado: 75.24752475247524.


## SDDP y Learning: stochastic cutting plane

En la formulación anterior, se asume que conocemos de antemano todos los ruidos posibles para calcular el promedio. ¿Qué ocurre si nos revelan realizaciones de los ruidos de a una muestra?

**Stochastic cutting plane:**

1. Se realiza la pasada forward, generando una muestra nueva de ruido $w^{(l)}$ y un punto de exploración $x_1^{(l)}$.

2. Para todas las muestras de ruido observadas $\{w^{(j)}, j=1,\ldots,l\}$ se resuelve el problema de 2a. etapa, generando un *corte aproximado* en $x_1^{(l)}$ promediando los hiperplanos.

3. Se "apaga" la aproximación anterior $V^{(l-1)}$ por un factor $\frac{l-1}{l}$.

4. Se agrega el nuevo corte, construyendo $V^{(l)}$.

In [8]:
#array de vectores de cuts. Arranca en la lower bound
cuts = [[0.0;0.0]];

#aca guardo solo los multiplicadores (para ver lo que pasa)
duals = [0.0];

#aca guardo los ruidos que fueron saliendo
noises = Array{Float64}(undef,0);

#condicion inicial de stock
stock_ini=0.0;

#inicializo la animacion para ir viendo los cortes paso a paso
anim=Animation()

@showprogress 1 "Computing..." for l=1:200
    
    #Forward Step (IDEM SDDP)
    model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))
    @variable(model,u0>=0);
    @variable(model,x0>=0);
    @variable(model,z);

    for i=1:length(cuts)
        cut=cuts[i]
        @constraint(model,z>=cut[1]+cut[2]*(x0+u0));
    end

    fix_x = @constraint(model,x0.==stock_ini);

    @objective(model,Min,p*u0+z);

    optimize!(model)

    x1_value=value.(u0)+value.(x0);


    #Sorteo el ruido del paso
    w=rand(DiscreteUniform(0,100));
    
    #lo agrego a los observados
    push!(noises,w)

    local_cuts = [];

    #resuelvo para cada ruido observado
    for k=1:length(noises)

        model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))

        @variable(model,u1>=0);
        fix_x = @constraint(model,x1_value-noises[k]+u1.>=0);

        @objective(model,Min,q*u1);

        optimize!(model)

        beta = objective_value(model);
        lambda = -dual.(fix_x)
        
        #guardo el lambda
        push!(duals,lambda);
        
        push!(local_cuts,[beta-lambda*x1_value;lambda])

    end

    #Apago los cortes anteriores
    cuts = (l-1)/l*cuts;
    
    #construyo el nuevo corte y lo agrego
    new_cut = mean(local_cuts);
    push!(cuts,new_cut);
        
    #Grafico para la animacion
    x=(0:100)
    pl=plot(;xlabel="Iteracion $l",legend=:none,xlim=(0,100),ylim=(-10,100))
    plot!(pl,x,V.(x,q,θ),lw=2)
    plot!(pl,x,piecewise_linear.(x,Ref(cuts)),lw=2)
    plot!(pl,x,piecewise_linear.(x,Ref([cuts[end]])),ls=:dash)
    frame(anim)

end

LoadError: UndefVarError: V not defined

In [9]:
gif(anim,"tmp2.gif",fps=10)

LoadError: ArgumentError: Cannot build empty animations

### Resuelvo el problema de stock

Con la aproximación de $V$ obtenida puedo resolver el problema forward una última vez para hallar el control óptimo:

In [10]:
model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))
@variable(model,u0>=0);
@variable(model,x0>=0);
@variable(model,z);

for i=1:length(cuts)
    cut=cuts[i]
    @constraint(model,z>=cut[1]+cut[2]*(x0+u0));
end

fix_x = @constraint(model,x0.==stock_ini);

@objective(model,Min,p*u0+z);

optimize!(model)

x1_value=value.(u0)+value.(x0);
println("Stock day ahead: $x1_value.")
println("Costo estimado: $(objective_value(model)).")


Stock day ahead: 5.0.
Costo estimado: 5.0.


### Precios duales

Una observación interesante surge de mirar lo que ocurre con los valores duales que fue calculando el algoritmo:

In [11]:
duals

2-element Vector{Float64}:
  0.0
 -2.0

### Dualidad del problema spot

Recordemos el problema spot:

$$\min_{u_1\geqslant 0} q u_1$$

sujeto a:

$$x_1 + u_1 \geqslant w,$$

Calculemos:

$$\mathcal{L}(u_1,\lambda) = q u_1 - \lambda (x_1+u_1-w) = (q-\lambda)u_1 + \lambda(w-x_1).$$

Minimizando el Lagrangeano en $u_1$ obtenemos el dual:

$$\mathcal{D}(\lambda) = \min_{u_1\geqslant 0} \mathcal{L}(u_1,\lambda) = \begin{cases} \lambda(w-x_1) &\quad \lambda\leqslant q \\ -\infty &\quad \lambda>q \end{cases}$$

Por lo tanto, el problema dual, *para todos los ruidos*, es lineal y tiene conjunto factible $[0,q]$. Sus extremos son $0$ y $q$ por lo que estas son las soluciones posibles.

De hecho, resolviendo el dual:

$$\arg\max_{0\leqslant \lambda \leqslant q} \mathcal{D}(\lambda) = \begin{cases} 0 &\quad w\leqslant x_1 \\ q &\quad w>x_1 \end{cases}$$

Entonces la construcción del hiperplano actual se reduce a: mirar para todos los ruidos si me quedé corto o largo, asignarle costo $q(w-x)^+$ a cada uno y promediar. La pendiente promedio es $q$ por la fracción de veces que me quedé corto.

## Stochastic Decomposition

**Idea:** aprovechar la estructura anterior para evitar resolver problemas lineales extra.

**Stochastic decomposition:**

1. Se realiza la pasada forward, generando una muestra nueva de ruido $w^{(l)}$ y un punto de exploración $x_1^{(l)}$.

2. Para la nueva muestra $w^{(l)}$, se resuelve el problema de 2a. etapa y se genera un nuevo dual $\lambda^{(l)}$. Este se agrega al conjunto de duales observados.

3. Para todas las muestras de ruido observadas hasta el momento $\{w^{(j)}, j=1,\ldots,l\}$ se evalúa $D(\lambda)$ en $x^{(l)}$ para ese $w^{(j)}$ y todos los $\lambda$ observados y nos quedamos con el máximo en cada uno.

4. Promediando los valores anteriores, generamos un *corte aproximado* en $x_1^{(l)}$ promediando los hiperplanos.

3. Se "apaga" la aproximación anterior $V^{(l-1)}$ por un factor $\frac{l-1}{l}$.

4. Se agrega el nuevo corte, construyendo $V^{(l)}$.

In [12]:
#array de vectores de cuts. Arranca en la lower bound
cuts = [[0.0;0.0]];

#aca guardo solo los multiplicadores (para ver lo que pasa)
duals = Set{Float64}();

#aca guardo los ruidos que fueron saliendo
noises = Array{Float64}(undef,0);

#condicion inicial de stock
stock_ini=0.0;

#inicializo la animacion para ir viendo los cortes paso a paso
anim=Animation()

@showprogress 1 "Computing..." for l=1:200
    
    #Forward Step (IDEM SDDP)
    model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))
    @variable(model,u0>=0);
    @variable(model,x0>=0);
    @variable(model,z);

    for i=1:length(cuts)
        cut=cuts[i]
        @constraint(model,z>=cut[1]+cut[2]*(x0+u0));
    end

    fix_x = @constraint(model,x0.==stock_ini);

    @objective(model,Min,p*u0+z);

    optimize!(model)

    x1_value=value.(u0)+value.(x0);


    #Sorteo el ruido del paso
    w=rand(DiscreteUniform(0,100));
    
    #lo agrego a los observados
    push!(noises,w)

    ##Resuelvo SOLO para el nuevo ruido
    model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))

    @variable(model,u1>=0);
    fix_x = @constraint(model,x1_value-w+u1.>=0);

    @objective(model,Min,q*u1);

    optimize!(model)

    beta = objective_value(model);
    lambda = dual.(fix_x)
        
    #Guardo el lambda en el conjunto observado
    push!(duals,lambda);
        
    
    #Obtengo el valor de costo y dual para cada ruido anterior buscando en el set de duals y genero un nuevo corte
    
    local_cuts = [];
    pik = similar(noises);
    #genero el new cut recorriendo todos los ruidos
    for j=1:length(noises)
        aux = [(pi*(noises[j]-x1_value),pi) for pi in duals]
        _, ix = findmax([a[1] for a in aux])
        pik[j] = aux[ix][2];
    end

    new_cut  = [mean(pik.*noises); -mean(pik) ];
    
    #Apago los cortes anteriores
    cuts = (l-1)/l*cuts;
    
    #agrego el nuevo corte
    push!(cuts,new_cut);
        
    #Grafico para la animacion
    x=(0:100)
    pl=plot(;xlabel="Iteracion $l",legend=:none,xlim=(0,100),ylim=(-10,100))
    plot!(pl,x,V.(x,q,θ),lw=2)
    plot!(pl,x,piecewise_linear.(x,Ref(cuts)),lw=2)
    plot!(pl,x,piecewise_linear.(x,Ref([cuts[end]])),ls=:dash)
    frame(anim)

end

LoadError: UndefVarError: V not defined

In [13]:
gif(anim,"tmp3.gif",fps=10)

LoadError: ArgumentError: Cannot build empty animations

### Resuelvo el problema de stock

Con la aproximación de $V$ obtenida puedo resolver el problema forward una última vez para hallar el control óptimo:

In [14]:
model = JuMP.Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env), "OutputFlag" => 0))
@variable(model,u0>=0);
@variable(model,x0>=0);
@variable(model,z);

for i=1:length(cuts)
    cut=cuts[i]
    @constraint(model,z>=cut[1]+cut[2]*(x0+u0));
end

fix_x = @constraint(model,x0.==stock_ini);

@objective(model,Min,p*u0+z);

optimize!(model)

x1_value=value.(u0)+value.(x0);
println("Stock day ahead: $x1_value.")
println("Costo estimado: $(objective_value(model)).")


Stock day ahead: 21.0.
Costo estimado: 21.0.


In [15]:
println("Duales observados: $duals")

Duales observados: Set([2.0])


## Preguntas:

* ¿Podemos usar estas ideas para generar variantes "learning" de SDDP?
* ¿Cómo escribimos bien el problema decisión-azar en el caso multi-etapa?
* ¿Cuál sería la generalización de SCP a multi-stage?
* ¿Cómo sería SCP en el caso azar-decisión?

### Intentos de respuesta y algunos caveats...(work in progress)